<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/crawl_data/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psycopg2

In [1]:
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchain-ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


In [11]:
import requests
from bs4 import BeautifulSoup as bp
import time
import csv
import json
import geocoder

In [ ]:
conn = psycopg2.connect(database = "travellocations",
                        user = "postgres",
                        host= 'localhost',
                        password = "cong2006",
                        port = 5432)

In [ ]:
cur = conn.cursor()

# Thực thi các lệnh SQL để tạo schema, loại dữ liệu, và bảng
cur.execute("""
DROP SCHEMA IF EXISTS places CASCADE;
CREATE SCHEMA IF NOT EXISTS places;
SET search_path TO places;

CREATE TYPE address AS (
    house_number TEXT,
    street TEXT,
    ward TEXT,
    district TEXT,
    city TEXT
);

CREATE TYPE location AS (
    latitude DECIMAL(9, 6),
    longitude DECIMAL(9, 6)
);

CREATE TABLE places.hotels (
    hotel_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    address address,
    location location,
    rating DECIMAL(2, 1),
    description TEXT,
    img_url JSON,
    comments TEXT
);

CREATE TABLE places.hotel_price_range (
    id SERIAL PRIMARY KEY,
    hotel_id INT REFERENCES places.hotels(hotel_id) ON DELETE CASCADE,
    room_type VARCHAR(100),
    occupancy INT,
    price DECIMAL(10, 2)
);
""")

# Cam kết các thay đổi vào cơ sở dữ liệu
conn.commit()

# # Đóng con trỏ và kết nối
cur.close()
# conn.close()


In [ ]:
            # "house_number": "...",
            # "street": "...",
            # "ward": "...",
            # "district": "...",
            # "city": "..."

hotel_name = 'Grand Hotel'
house_number = '123'
address_street = 'Main St'
address_ward = 'Main ward'
address_district = 'Downtown'
address_city = 'New York'
latitude = 40.712776
longitude = -74.005974
rating = 4.5
description = 'A luxury hotel in the heart of the city.'
img_url =   '{"img_1": "http://example.com/image1.jpg", "img_2": "http://example.com/image2.jpg"}'
comments = 'Great amenities and service.'
room_type = 'Suite'
occupancy = 2
price = 299.99
cur = conn.cursor()

cur.execute("""
    INSERT INTO places.hotels (name, address, location, rating, description, img_url, comments)
    VALUES (%s, ROW(%s, %s, %s, %s, %s), ROW(%s, %s), %s, %s, %s, %s)
    RETURNING hotel_id;
""", (hotel_name, house_number, address_street, address_ward, address_district, address_city, latitude, longitude, rating, description, img_url, comments))

# Lấy ID của khách sạn mới chèn
hotel_id = cur.fetchone()[0]

# Chèn dữ liệu vào bảng hotel_price_range
cur.execute("""
    INSERT INTO places.hotel_price_range (hotel_id, room_type, occupancy, price)
    VALUES (%s, %s, %s, %s);
""", (hotel_id, room_type, occupancy, price))

# Cam kết các thay đổi vào cơ sở dữ liệu
conn.commit()

# # Đóng con trỏ và kết nối
cur.close()
# conn.close()

In [5]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'en-US, en;q=0.5',
}

In [ ]:
!pip install -qU langchain-google-genai

In [ ]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 12030    0 12030    0     0  42110      0 --:--:-- --:--:-- --:--:-- 42062
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [7]:
import subprocess
import time
import threading

# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")

Ollama server is running in the background


In [37]:
!ollama pull llama3:latest

[GIN] 2024/08/18 - 05:22:56 | 200 |      29.284µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  29 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 8.5 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  31 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  62 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏  70 MB/4.7 GB                  pul

In [38]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3:latest", temperature=0)

In [39]:
def get_page_html(url):
    res = requests.get(url=url, headers=REQUEST_HEADER)
    return res.text

def get_hotel_price(soup):
    price_element = soup.find('div', attrs={'style': 'color: rgb(255, 94, 31); font-size: 20px;'})
    if price_element:
        true_price = price_element.text.strip().replace('VND', '').replace('.', '')
        return float(true_price)
    return None

def get_hotel_name(soup):
    name = soup.find('div', class_='css-901oao r-a5wbuh r-1enofrn r-b88u0q r-1cwl3u0 r-fdjqy7 r-3s2u2q')
    return name.text.strip() if name else None

def get_hotel_rating(soup):
    rating = soup.find('div', class_='css-901oao r-jwli3a r-a5wbuh r-s67bdx r-b88u0q r-10cxs7j r-q4m81j')
    return rating.text.strip() if rating else None

def get_hotel_des(soup):
    des = soup.find('div', attrs={'style': 'font-family:Godwit, -apple-system, BlinkMacSystemFont, Segoe UI, Roboto, Arial, sans-serif, Apple Color Emoji, Segoe UI Emoji, Segoe UI Symbol;font-size:14px;line-height:20px;max-height:80px;overflow:hidden'})
    return des.text.strip().replace('\n', '') if des else None

def get_hotel_address(soup):
    address = soup.find('div', class_='css-901oao css-cens5h r-13awgt0 r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
    if address:
        address_full = address.text.strip().replace('\t', '')

        prompt = f"""
            Separate the following address into 5 parts: house number, street name, ward name, district name, and city name:
            {address_full}

            Provide the result in Vietnamese and strictly in the following format:
            {{
            "house_number": "...",
            "street": "...",
            "ward": "...",
            "district": "...",
            "city": "..."
            }}
            """
        response = llm.invoke(prompt)
        response_text = str(response.content)
        cleaned_json_str = response_text.strip('```json\n').strip('```\n')
        print(cleaned_json_str)
        try:
            result_dict = json.loads(cleaned_json_str)
            return result_dict
        except json.JSONDecodeError:
            return {'details': '', 'ward': '', 'district': '', 'city': ''}

def get_hotel_location(address):
    addr = str(address['ward'] + address['district'] + address['city'])
    g = geocoder.osm(addr)
    location = g.osm
    latitude = location['y']
    longitude = location['x']

    return {'latitude': latitude, 'longitude': longitude}

def get_hotel_comments(soup):
    comments = []
    a = soup.findAll('div', class_='css-901oao css-cens5h r-cwxd7f r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
    for comment in a:
        comments.append(comment.text.strip())
    return comments

def insert_hotel_data(conn, info):
    cur = conn.cursor()
    try:
        cur.execute("""
            INSERT INTO places.hotels (name, address, location, rating, description, img_url, comments)
            VALUES (%s, ROW(%s, %s, %s, %s), ROW(%s, %s), %s, %s, %s, %s)
            RETURNING hotel_id;
        """, (
            info['name'],
            info['address']['details'],
            info['address']['ward'],
            info['address']['district'],
            info['address']['city'],
            info['location']['latitude'],
            info['location']['longitude'],
            info['rating'],
            info['description'],
            info.get('img_url', None),
            info['comments']
        ))

        hotel_id = cur.fetchone()[0]

        if 'price' in info:
            cur.execute("""
                INSERT INTO places.hotel_price_range (hotel_id, room_type, occupancy, price)
                VALUES (%s, %s, %s, %s)
            """, (
                hotel_id,
                info.get('room_type', None),
                info.get('occupancy', None),
                info['price']
            ))

        conn.commit()
    except Exception as e:
        print(f"Error inserting data: {e}")
        conn.rollback()
    finally:
        cur.close()

def extract_hotels_info(url):
    info = {}
    html = get_page_html(url)
    soup = bp(html, 'lxml')
    info['name'] = get_hotel_name(soup)
    info['price'] = get_hotel_price(soup)
    info['rating'] = get_hotel_rating(soup)
    info['address'] = get_hotel_address(soup)
    info['location'] = get_hotel_location(info['address'])
    info['description'] = get_hotel_des(soup)
    info['comments'] = get_hotel_comments(soup)
    return info


In [ ]:
db_params = {
    'database': 'travellocations',
    'user': 'postgres',
    'host': 'localhost',
    'password': 'cong2006',
    'port': 5432
}

with open('hotels.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        url = row[0]
        data = extract_hotels_info(url)
        insert_hotel_data(conn, data)
        time.sleep(0.25)

# conn.close()

In [28]:
url = "https://www.traveloka.com/vi-vn/hotel/vietnam/hanoi-sunshine-hotel-1000000468475?spec=18-08-2024.19-08-2024.1.1.HOTEL.1000000468475..2"
html = get_page_html(url)
soup = bp(html, 'lxml')
address = get_hotel_address(soup)
print(address)

[GIN] 2024/08/18 - 05:15:46 | 200 |  1.247051516s |       127.0.0.1 | POST     "/api/chat"
{
"chi tiết": "18 Hàng Hòm",
"phường": "Hàng Gai",
"quận": "Quận Hoàn Kiếm",
"thành phố": "Hà Nội"
} 
{'chi tiết': '18 Hàng Hòm', 'phường': 'Hàng Gai', 'quận': 'Quận Hoàn Kiếm', 'thành phố': 'Hà Nội'}


In [42]:
url = "https://www.traveloka.com/vi-vn/hotel/vietnam/hanoi-sunshine-hotel-1000000468475?spec=18-08-2024.19-08-2024.1.1.HOTEL.1000000468475..2"
html = get_page_html(url)
soup = bp(html, 'lxml')
address = soup.find('div', class_='css-901oao css-cens5h r-13awgt0 r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
address_full = address.text.strip().replace('\t', '')
print(address_full)
prompt = f"""
    Separate the following address into 5 parts: house number, street name, ward name, district name, and city name:
            {address_full}

            Provide the result in Vietnamese and strictly in the following format:
            {{
            "house_number": "...",
            "street": "...",
            "ward": "...",
            "district": "...",
            "city": "..."
            }}
    """
response = llm.invoke(prompt)
print(response.content)


18 Hàng Hòm, Hàng Gai, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 100000
[GIN] 2024/08/18 - 05:26:39 | 200 |  2.486032381s |       127.0.0.1 | POST     "/api/chat"
Here is the separated address in the requested format:

{
"house_number": "18",
"street": "Hàng Hòm",
"ward": "Hàng Gai",
"district": "Hoàn Kiếm",
"city": "Hà Nội"
}

Note: The district name "Quận" is not included as it's a Vietnamese word for "district", and the actual district name is "Hoàn Kiếm".


In [ ]:
address = get_hotel_address(soup)
print(address)
addr = str("phường "+address['ward']+", quận "+address['district']+", thành phố "+address['city'])
print(addr)
g = geocoder.osm(addr)
location = g.osm
location
print(type(location))
latitude = location['y']
longitude = location['x']
print(latitude,longitude)

In [ ]:
!pip install geocoder

I0000 00:00:1723934514.000823   30586 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [ ]:
import geocoder
g = geocoder.osm('Hàng Gai, Hoàn Kiếm, Hà Nội, Việt Nam')
g.osm

{'x': 105.8490719,
 'y': 21.0321611,
 'addr:city': 'Thành phố Hà Nội',
 'addr:country': 'Việt Nam',
 'addr:postal': '11015'}